<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Frequency-Encoding" data-toc-modified-id="Frequency-Encoding-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Frequency Encoding</a></span><ul class="toc-item"><li><span><a href="#Single-processing" data-toc-modified-id="Single-processing-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Single processing</a></span></li><li><span><a href="#Multiple-processing" data-toc-modified-id="Multiple-processing-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Multiple processing</a></span><ul class="toc-item"><li><span><a href="#multiprocessing.pool" data-toc-modified-id="multiprocessing.pool-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>multiprocessing.pool</a></span></li><li><span><a href="#concurrent.futuresProcess.PoolExecutor" data-toc-modified-id="concurrent.futuresProcess.PoolExecutor-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>concurrent.futuresProcess.PoolExecutor</a></span></li></ul></li></ul></li><li><span><a href="#Label-Encoding" data-toc-modified-id="Label-Encoding-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Label Encoding</a></span><ul class="toc-item"><li><span><a href="#Single-Processing" data-toc-modified-id="Single-Processing-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Single Processing</a></span></li></ul></li><li><span><a href="#Average-Encoding" data-toc-modified-id="Average-Encoding-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Average Encoding</a></span><ul class="toc-item"><li><span><a href="#Single-Processing" data-toc-modified-id="Single-Processing-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Single Processing</a></span></li><li><span><a href="#Multiple-Processing" data-toc-modified-id="Multiple-Processing-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Multiple Processing</a></span><ul class="toc-item"><li><span><a href="#itertools.product-&amp;-multiprocessing.pool" data-toc-modified-id="itertools.product-&amp;-multiprocessing.pool-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>itertools.product &amp; multiprocessing.pool</a></span></li><li><span><a href="#itertools.product-&amp;-concurrent.futureProcess.ProcessPoolExecutor" data-toc-modified-id="itertools.product-&amp;-concurrent.futureProcess.ProcessPoolExecutor-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>itertools.product &amp; concurrent.futureProcess.ProcessPoolExecutor</a></span></li></ul></li></ul></li></ul></div>

# Load Data

In [1]:
import os
import gc
import time
import pickle
import pandas as pd
import numpy as np
import itertools
from multiprocessing.pool import Pool
from concurrent.futures import ProcessPoolExecutor

import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"  

In [2]:
PROJECT_DIR = os.path.abspath('.')
DATA_DIR = os.path.join(PROJECT_DIR, 'data')

In [3]:
with open(os.path.join(DATA_DIR, 'train.pkl'), 'rb') as f:
    train_df_ = pickle.load(f)
    
with open(os.path.join(DATA_DIR, 'test.pkl'), 'rb') as f:
    test_df_ = pickle.load(f)

In [4]:
train_df_.head(3)

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000.0,86400.0,68.5,W,13926.0,327.0,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001.0,86401.0,29.0,W,2755.0,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002.0,86469.0,59.0,W,4663.0,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
test_df_.head(3)

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
0,3663549.0,18403224.0,31.950001,W,10409.0,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550.0,18403264.0,49.000000,W,4272.0,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3663551.0,18403310.0,171.000000,W,4476.0,574.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
str_type = ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5',
            'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30',
            'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']

In [7]:
# train_df_[['addr1', 'card1', 'card2', 'card3', 'P_emaildomain']].head()

# Frequency Encoding

In [8]:
# frequency encode
def encode_FE(df1, df2, cols):
    for col in cols:
        df = pd.concat([df1[col], df2[col]])
        vc = df.value_counts(dropna=True, normalize=True).to_dict()
        vc[-1] = -1
        nm = col + "FE"
        df1[nm] = df1[col].map(vc)
        df1[nm] = df1[nm].astype("float32")
        df2[nm] = df2[col].map(vc)
        df2[nm] = df2[nm].astype("float32")
        print(col)

In [9]:
fea_cols = ['addr1', 'card1', 'card2', 'card3', 'P_emaildomain']

## Single processing

In [10]:
train_df = train_df_.copy()
test_df = test_df_.copy()


start = time.time()
encode_FE(train_df, test_df, cols=fea_cols)
elapsed = (time.time() - start)
print("Time used:",elapsed)


train_df[['addr1FE', 'card1FE', 'card2FE', 'card3FE', 'P_emaildomainFE']].head()

addr1
card1
card2
card3
P_emaildomain
Time used: 0.39342308044433594


,addr1FE,card1FE,card2FE,card3FE,P_emaildomainFE
0,0.044554,0.000051,0.008551,0.87566,NaN
1,0.079616,0.001219,0.005197,0.87566,0.466807
2,0.050094,0.001635,0.065110,0.87566,0.010641
3,0.018071,0.006958,0.010425,0.87566,0.195788
4,0.007358,0.000027,0.025710,0.87566,0.466807


## Multiple processing

In [11]:
# frequency encode
def sin_encode_FE(df1, df2, col):
    new_train_df = pd.DataFrame({})
    new_test_df = pd.DataFrame({})
    df = pd.concat([df1[col], df2[col]])
    vc = df.value_counts(dropna=True, normalize=True).to_dict()
    vc[-1] = -1
    nm = col + "FE"
    new_train_df[nm] = df1[col].map(vc)
    new_train_df[nm] = new_train_df[nm].astype("float32")
    new_test_df[nm] = df2[col].map(vc)
    new_test_df[nm] = new_test_df[nm].astype("float32")
    print(col)
    return new_train_df, new_test_df

    
def multi_encode_FE(arg):
    new_train_df, new_test_df = sin_encode_FE(*arg)
    return new_train_df, new_test_df

### multiprocessing.pool

In [12]:
train_df = train_df_.copy()
test_df = test_df_.copy()

ts = time.time()

tasks = []
for col in fea_cols:
    tasks.append((train_df[[col]], test_df[[col]], col))

pool_num = 7 if len(fea_cols) < 8 else 7
with Pool(pool_num) as p:
    new_res = p.map(multi_encode_FE, tasks)

for tr, te in new_res:
    train_df = pd.concat([train_df, tr], axis=1)
    test_df = pd.concat([test_df, te], axis=1)
    
elapsed = (time.time() - start)
print("Time used:",elapsed)

train_df[['addr1FE', 'card1FE', 'card2FE', 'card3FE', 'P_emaildomainFE']].head()

addr1
card2
card3
card1
P_emaildomain
Time used: 46.516693115234375


,addr1FE,card1FE,card2FE,card3FE,P_emaildomainFE
0,0.044554,0.000051,0.008551,0.87566,NaN
1,0.079616,0.001219,0.005197,0.87566,0.466807
2,0.050094,0.001635,0.065110,0.87566,0.010641
3,0.018071,0.006958,0.010425,0.87566,0.195788
4,0.007358,0.000027,0.025710,0.87566,0.466807


### concurrent.futuresProcess.PoolExecutor

In [13]:
train_df = train_df_.copy()
test_df = test_df_.copy()


start = time.time()
tasks = []
for col in fea_cols:
    tasks.append((train_df[[col]], test_df[[col]], col))
    
    
process_num = 7 if len(fea_cols) < 8 else 7
with ProcessPoolExecutor(max_workers=process_num) as excutor:
    new_res = excutor.map(multi_encode_FE, tasks)


for tr, te in new_res:
    train_df = pd.concat([train_df, tr], axis=1)
    test_df = pd.concat([test_df, te], axis=1)

    
elapsed = (time.time() - start)
print("Time used:",elapsed)


train_df[['addr1FE', 'card1FE', 'card2FE', 'card3FE', 'P_emaildomainFE']].head()

card1
card2
addr1
card3
P_emaildomain
Time used: 20.001041889190674


,addr1FE,card1FE,card2FE,card3FE,P_emaildomainFE
0,0.044554,0.000051,0.008551,0.87566,NaN
1,0.079616,0.001219,0.005197,0.87566,0.466807
2,0.050094,0.001635,0.065110,0.87566,0.010641
3,0.018071,0.006958,0.010425,0.87566,0.195788
4,0.007358,0.000027,0.025710,0.87566,0.466807


# Label Encoding

In [14]:
# label encode
def encode_LE(col, train, test, verbose=True):
    df_comb = pd.concat([train[col], test[col]], axis=0)
    df_comb, _ = pd.factorize(df_comb)
    nm = col
    if df_comb.max() > 32000:
        train[nm] = df_comb[0: len(train)].astype("float32")
        test[nm] = df_comb[len(train):].astype("float32")
    else:
        train[nm] = df_comb[0: len(train)].astype("float16")
        test[nm] = df_comb[len(train):].astype("float16")
    del df_comb
    gc.collect()
    if verbose:
        print(col)

        
# COMBINE FEATURES
def encode_CB(col1, col2, df1, df2):
    nm = col1 + '_' + col2
    df1[nm] = df1[col1].astype(str) + '_' + df1[col2].astype(str)
    df2[nm] = df2[col1].astype(str) + '_' + df2[col2].astype(str)
    encode_LE(nm, df1, df2, verbose=False)
    print(nm, ', ', end='')

## Single Processing

In [15]:
train_df2 = train_df.copy()
test_df2 = test_df.copy()


start = time.time()
encode_CB('card1', 'addr1', train_df2, test_df2)
encode_CB('card1_addr1', 'P_emaildomain', train_df2, test_df2)
elapsed = (time.time() - start)
print("Time used:",elapsed)


train_df2[['card1', 'addr1', 'card1_addr1']].head()
train_df2[['card1_addr1', 'P_emaildomain', 'card1_addr1_P_emaildomain']].head()

card1_addr1 , card1_addr1_P_emaildomain , Time used: 2.419175863265991


,card1,addr1,card1_addr1
0,13926.0,315.0,0.0
1,2755.0,325.0,1.0
2,4663.0,330.0,2.0
3,18132.0,476.0,3.0
4,4497.0,420.0,4.0


,card1_addr1,P_emaildomain,card1_addr1_P_emaildomain
0,0.0,NaN,-1.0
1,1.0,gmail.com,0.0
2,2.0,outlook.com,1.0
3,3.0,yahoo.com,2.0
4,4.0,gmail.com,3.0


In [16]:
train_df3 = train_df2.copy()
test_df3 = test_df2.copy()
fea_cols = ['card1_addr1', 'card1_addr1_P_emaildomain']


start = time.time()
encode_FE(train_df3, test_df3, fea_cols)
elapsed = (time.time() - start)
print("Time used:",elapsed)


train_df3[['card1_addr1', 'card1_addr1_P_emaildomain']].head()
train_df3 = train_df2.copy()
test_df3 = test_df2.copy()


start = time.time()
tasks = []
for col in fea_cols:
    tasks.append((train_df3[[col]], test_df3[[col]], col))

    
process_num = 7 if len(fea_cols) < 8 else 7
with ProcessPoolExecutor(max_workers=process_num) as excutor:
    new_res = excutor.map(multi_encode_FE, tasks)
    # new_res: python generator
    

for tr, te in new_res:
    train_df3 = pd.concat([train_df3, tr], axis=1)
    test_df3 = pd.concat([test_df3, te], axis=1)
elapsed = (time.time() - start)
print("Time used:",elapsed)


train_df3[['card1_addr1', 'card1_addr1_P_emaildomain']].head()

card1_addr1
card1_addr1_P_emaildomain
Time used: 0.23212313652038574


,card1_addr1,card1_addr1_P_emaildomain
0,0.0,-1.0
1,1.0,0.0
2,2.0,1.0
3,3.0,2.0
4,4.0,3.0


card1_addr1
card1_addr1_P_emaildomain
Time used: 9.629117965698242


,card1_addr1,card1_addr1_P_emaildomain
0,0.0,-1.0
1,1.0,0.0
2,2.0,1.0
3,3.0,2.0
4,4.0,3.0


# Average Encoding

In [17]:
def encode_AG(main_columns, uids, df_train, df_test, aggregations=["mean"], fillna=True, usena=False):
    for main_column in main_columns:
        print(f"\nmain_column: {main_column}")
        for col in uids:
            print(f"col: {col}")
            for agg_type in aggregations:
                print(f"agg_type: {agg_type}")
                new_column = main_column + "_" + col + "_" + agg_type
                print(f"new_column: {new_column}")
                temp_df = pd.concat([df_train[[col, main_column]], df_test[[col, main_column]]])
                if usena:
                    temp_df.loc[temp_df[main_column] == -1, main_column] = np.nan


                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                    columns={agg_type: new_column})
                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_column].to_dict()
                df_train[new_column] = df_train[col].map(temp_df).astype("float32")
                df_test[new_column] = df_test[col].map(temp_df).astype("float32")
                if fillna:
                    df_train[new_column].fillna(-1, inplace=True)
                    df_test[new_column].fillna(-1, inplace=True)

In [18]:
new_cols = [
    'TransactionAmt_card1_mean',
    'TransactionAmt_card1_std',
    'TransactionAmt_card1_addr1_mean',
    'TransactionAmt_card1_addr1_std',
    'TransactionAmt_card1_addr1_P_emaildomain_mean',
    'TransactionAmt_card1_addr1_P_emaildomain_std',
    'D9_card1_mean',
    'D9_card1_std',
    'D9_card1_addr1_mean',
    'D9_card1_addr1_std',
    'D9_card1_addr1_P_emaildomain_mean',
    'D9_card1_addr1_P_emaildomain_std',
    'D11_card1_mean',
    'D11_card1_std',
    'D11_card1_addr1_mean',
    'D11_card1_addr1_std',
    'D11_card1_addr1_P_emaildomain_mean',
    'D11_card1_addr1_P_emaildomain_std'
]

## Single Processing

In [19]:
train_df4 = train_df3.copy()
test_df4 = test_df3.copy()


start = time.time()
encode_AG(
    ['TransactionAmt', 'D9', 'D11'], 
    ['card1', 'card1_addr1', 'card1_addr1_P_emaildomain'], 
    train_df4, test_df4,
    ['mean', 'std'],
    usena=False
)
elapsed = (time.time() - start)
print("Time used:",elapsed)


train_df4[new_cols].head()


main_column: TransactionAmt
col: card1
agg_type: mean
new_column: TransactionAmt_card1_mean
agg_type: std
new_column: TransactionAmt_card1_std
col: card1_addr1
agg_type: mean
new_column: TransactionAmt_card1_addr1_mean
agg_type: std
new_column: TransactionAmt_card1_addr1_std
col: card1_addr1_P_emaildomain
agg_type: mean
new_column: TransactionAmt_card1_addr1_P_emaildomain_mean
agg_type: std
new_column: TransactionAmt_card1_addr1_P_emaildomain_std

main_column: D9
col: card1
agg_type: mean
new_column: D9_card1_mean
agg_type: std
new_column: D9_card1_std
col: card1_addr1
agg_type: mean
new_column: D9_card1_addr1_mean
agg_type: std
new_column: D9_card1_addr1_std
col: card1_addr1_P_emaildomain
agg_type: mean
new_column: D9_card1_addr1_P_emaildomain_mean
agg_type: std
new_column: D9_card1_addr1_P_emaildomain_std

main_column: D11
col: card1
agg_type: mean
new_column: D11_card1_mean
agg_type: std
new_column: D11_card1_std
col: card1_addr1
agg_type: mean
new_column: D11_card1_addr1_mean
agg_

,TransactionAmt_card1_mean,TransactionAmt_card1_std,TransactionAmt_card1_addr1_mean,TransactionAmt_card1_addr1_std,TransactionAmt_card1_addr1_P_emaildomain_mean,TransactionAmt_card1_addr1_P_emaildomain_std,D9_card1_mean,D9_card1_std,D9_card1_addr1_mean,D9_card1_addr1_std,D9_card1_addr1_P_emaildomain_mean,D9_card1_addr1_P_emaildomain_std,D11_card1_mean,D11_card1_std,D11_card1_addr1_mean,D11_card1_addr1_std,D11_card1_addr1_P_emaildomain_mean,D11_card1_addr1_P_emaildomain_std
0,316.570343,351.514008,142.000000,152.654022,118.096176,192.946793,0.708333,0.157288,-1.000000,-1.000000,0.576379,0.289567,9.571428,16.972246,13.000000,-1.000000,203.92926,226.701904
1,213.053818,391.543884,236.318405,499.708618,512.468567,996.724609,0.610897,0.322520,0.466666,0.355268,0.770833,0.029462,203.424271,249.957642,70.705879,125.586159,134.62500,269.789307
2,104.876938,130.380966,65.733833,54.568237,73.500000,17.058722,0.736111,0.024056,-1.000000,-1.000000,-1.000000,-1.000000,99.132904,159.472763,163.161285,203.674271,413.25000,71.714134
3,120.958702,196.463486,112.923775,211.492203,74.112679,57.115608,0.563248,0.313940,0.781250,0.157288,0.750000,-1.000000,202.725235,237.283981,215.582092,215.294296,206.52272,209.169174
4,99.811668,69.829735,50.000000,-1.000000,50.000000,-1.000000,0.624999,0.310689,-1.000000,-1.000000,-1.000000,-1.000000,204.333328,232.184769,-1.000000,-1.000000,-1.00000,-1.000000


## Multiple Processing

In [20]:
train_df4 = train_df3.copy()
test_df4 = test_df3.copy()


def sin_encode_AG(main_column, uid, agg_type, df_train=train_df4, df_test=test_df4, fillna=True, usena=False):
    print(f"\nmain_column: {main_column}, uid: {uid}, agg_type: {agg_type}")
    new_train = pd.DataFrame({})
    new_test = pd.DataFrame({})
    
    new_column = main_column + "_" + uid + "_" + agg_type
    print(f"\nnew_column: {new_column}")
    temp_df = pd.concat([df_train[[uid, main_column]], df_test[[uid, main_column]]])
    if usena:
        temp_df.loc[temp_df[main_column] == -1, main_column] = np.nan

    temp_df = temp_df.groupby([uid])[main_column].agg([agg_type]).reset_index().rename(
        columns={agg_type: new_column})
    temp_df.index = list(temp_df[uid])
    temp_df = temp_df[new_column].to_dict()
    new_train[new_column] = df_train[uid].map(temp_df).astype("float32")
    new_test[new_column] = df_test[uid].map(temp_df).astype("float32")
    if fillna:
        new_train[new_column].fillna(-1, inplace=True)
        new_test[new_column].fillna(-1, inplace=True)
    return new_train, new_test 


def multi_encode_AG(arg):
    new_train, new_test = sin_encode_AG(*arg)
    return new_train, new_test

### itertools.product & multiprocessing.pool

In [21]:
start = time.time()


main_columns = ['TransactionAmt', 'D9', 'D11']
uids = ['card1', 'card1_addr1', 'card1_addr1_P_emaildomain']
aggregations = ['mean', 'std']
tasks = list(itertools.product(main_columns, uids, aggregations))
    
    
pool_num = 7 if len(tasks) < 8 else 7
with Pool(pool_num) as p:
    new_res = p.map(multi_encode_AG, tasks)

    
for tr, te in new_res:
    train_df4 = pd.concat([train_df4, tr], axis=1)
    test_df4 = pd.concat([test_df4, te], axis=1)
elapsed = (time.time() - start)
print("Time used:",elapsed)


train_df4[new_cols].head()


main_column: TransactionAmt, uid: card1_addr1_P_emaildomain, agg_type: std
main_column: TransactionAmt, uid: card1, agg_type: mean
main_column: TransactionAmt, uid: card1, agg_type: std
main_column: TransactionAmt, uid: card1_addr1, agg_type: mean
main_column: D9, uid: card1, agg_type: mean
main_column: TransactionAmt, uid: card1_addr1, agg_type: std


main_column: TransactionAmt, uid: card1_addr1_P_emaildomain, agg_type: mean





new_column: TransactionAmt_card1_addr1_P_emaildomain_mean
new_column: TransactionAmt_card1_addr1_P_emaildomain_std
new_column: TransactionAmt_card1_mean
new_column: TransactionAmt_card1_addr1_std
new_column: TransactionAmt_card1_addr1_mean
new_column: TransactionAmt_card1_std
new_column: D9_card1_mean







main_column: D9, uid: card1, agg_type: std

new_column: D9_card1_std

main_column: D9, uid: card1_addr1, agg_type: mean


new_column: D9_card1_addr1_mean
main_column: D9, uid: card1_addr1, agg_type: std

new_column: D9_card1_addr1_std

main_column: D9, 

,TransactionAmt_card1_mean,TransactionAmt_card1_std,TransactionAmt_card1_addr1_mean,TransactionAmt_card1_addr1_std,TransactionAmt_card1_addr1_P_emaildomain_mean,TransactionAmt_card1_addr1_P_emaildomain_std,D9_card1_mean,D9_card1_std,D9_card1_addr1_mean,D9_card1_addr1_std,D9_card1_addr1_P_emaildomain_mean,D9_card1_addr1_P_emaildomain_std,D11_card1_mean,D11_card1_std,D11_card1_addr1_mean,D11_card1_addr1_std,D11_card1_addr1_P_emaildomain_mean,D11_card1_addr1_P_emaildomain_std
0,316.570343,351.514008,142.000000,152.654022,118.096176,192.946793,0.708333,0.157288,-1.000000,-1.000000,0.576379,0.289567,9.571428,16.972246,13.000000,-1.000000,203.92926,226.701904
1,213.053818,391.543884,236.318405,499.708618,512.468567,996.724609,0.610897,0.322520,0.466666,0.355268,0.770833,0.029462,203.424271,249.957642,70.705879,125.586159,134.62500,269.789307
2,104.876938,130.380966,65.733833,54.568237,73.500000,17.058722,0.736111,0.024056,-1.000000,-1.000000,-1.000000,-1.000000,99.132904,159.472763,163.161285,203.674271,413.25000,71.714134
3,120.958702,196.463486,112.923775,211.492203,74.112679,57.115608,0.563248,0.313940,0.781250,0.157288,0.750000,-1.000000,202.725235,237.283981,215.582092,215.294296,206.52272,209.169174
4,99.811668,69.829735,50.000000,-1.000000,50.000000,-1.000000,0.624999,0.310689,-1.000000,-1.000000,-1.000000,-1.000000,204.333328,232.184769,-1.000000,-1.000000,-1.00000,-1.000000


### itertools.product & concurrent.futureProcess.ProcessPoolExecutor

In [22]:
start = time.time()
   

main_columns = ['TransactionAmt', 'D9', 'D11']
uids = ['card1', 'card1_addr1', 'card1_addr1_P_emaildomain']
aggregations = ['mean', 'std']
tasks = list(itertools.product(main_columns, uids, aggregations))


process_num = 7 if len(tasks) < 8 else 7
with ProcessPoolExecutor(max_workers=process_num) as excutor:
    new_res = excutor.map(multi_encode_AG, tasks)

for tr, te in new_res:
    train_df4 = pd.concat([train_df4, tr], axis=1)
    test_df4 = pd.concat([test_df4, te], axis=1)
elapsed = (time.time() - start)
print("Time used:",elapsed)


train_df4[new_cols].head()


main_column: TransactionAmt, uid: card1, agg_type: std
main_column: TransactionAmt, uid: card1_addr1_P_emaildomain, agg_type: std
main_column: TransactionAmt, uid: card1_addr1_P_emaildomain, agg_type: mean
main_column: TransactionAmt, uid: card1_addr1, agg_type: mean



main_column: TransactionAmt, uid: card1, agg_type: mean

main_column: D9, uid: card1, agg_type: mean
main_column: TransactionAmt, uid: card1_addr1, agg_type: std
new_column: TransactionAmt_card1_std
new_column: TransactionAmt_card1_addr1_P_emaildomain_mean
new_column: TransactionAmt_card1_addr1_mean





new_column: TransactionAmt_card1_addr1_P_emaildomain_std

new_column: D9_card1_mean
new_column: TransactionAmt_card1_addr1_std

new_column: TransactionAmt_card1_mean



main_column: D9, uid: card1, agg_type: std

new_column: D9_card1_std

main_column: D9, uid: card1_addr1, agg_type: mean

new_column: D9_card1_addr1_mean

main_column: D9, uid: card1_addr1, agg_type: std

new_column: D9_card1_addr1_std

main_column: D9, 

,TransactionAmt_card1_mean,TransactionAmt_card1_mean,TransactionAmt_card1_std,TransactionAmt_card1_std,TransactionAmt_card1_addr1_mean,TransactionAmt_card1_addr1_mean,TransactionAmt_card1_addr1_std,TransactionAmt_card1_addr1_std,TransactionAmt_card1_addr1_P_emaildomain_mean,TransactionAmt_card1_addr1_P_emaildomain_mean,...,D11_card1_std,D11_card1_std,D11_card1_addr1_mean,D11_card1_addr1_mean,D11_card1_addr1_std,D11_card1_addr1_std,D11_card1_addr1_P_emaildomain_mean,D11_card1_addr1_P_emaildomain_mean,D11_card1_addr1_P_emaildomain_std,D11_card1_addr1_P_emaildomain_std
0,316.570343,316.570343,351.514008,351.514008,142.000000,142.000000,152.654022,152.654022,118.096176,118.096176,...,16.972246,16.972246,13.000000,13.000000,-1.000000,-1.000000,203.92926,203.92926,226.701904,226.701904
1,213.053818,213.053818,391.543884,391.543884,236.318405,236.318405,499.708618,499.708618,512.468567,512.468567,...,249.957642,249.957642,70.705879,70.705879,125.586159,125.586159,134.62500,134.62500,269.789307,269.789307
2,104.876938,104.876938,130.380966,130.380966,65.733833,65.733833,54.568237,54.568237,73.500000,73.500000,...,159.472763,159.472763,163.161285,163.161285,203.674271,203.674271,413.25000,413.25000,71.714134,71.714134
3,120.958702,120.958702,196.463486,196.463486,112.923775,112.923775,211.492203,211.492203,74.112679,74.112679,...,237.283981,237.283981,215.582092,215.582092,215.294296,215.294296,206.52272,206.52272,209.169174,209.169174
4,99.811668,99.811668,69.829735,69.829735,50.000000,50.000000,-1.000000,-1.000000,50.000000,50.000000,...,232.184769,232.184769,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,-1.00000,-1.000000,-1.000000


In [23]:
train_df5 = train_df4.copy()
test_df5 = test_df4.copy()

for col in str_type:
    try:
        encode_LE(col, train_df5, test_df5)
    except:
        print(f"KeyError: {col}")

ProductCD
card4
card6
P_emaildomain
R_emaildomain
M1
M2
M3
M4
M5
M6
M7
M8
M9
KeyError: id_12
KeyError: id_15
KeyError: id_16
KeyError: id_23
KeyError: id_27
KeyError: id_28
KeyError: id_29
KeyError: id_30
KeyError: id_31
KeyError: id_33
KeyError: id_34
KeyError: id_35
KeyError: id_36
KeyError: id_37
KeyError: id_38
DeviceType
DeviceInfo
